Importer les bases de données train et test 

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}

## Read csvs

train = gpd.read_file('./train.geojson', index_col=0)
test = gpd.read_file('./test.geojson', index_col=0)
train_y = train['change_type'].apply(lambda x: change_type_map[x])

travail sur le feature geometry, création de nouveaux features plus facile pour les calculs après : périmètre, surface, disance moyenne des points au centroid, variance des distances des points au centroid

In [ ]:
from shapely.geometry import Polygon, Point
import numpy as np

def calculate_area(polygon):
    area = polygon.area
    return area

def calculate_perimeter(polygon):
    perimeter = polygon.length
    return perimeter

def calculate_var_distance_to_center(polygon):
    center = polygon.centroid
    exterior_coords = polygon.exterior.coords #on enlève la dernière coordonnée puisque c'est le meme point que le premier
    distances = [center.distance(Point(x, y)) for x, y in exterior_coords]
    return np.var(distances)

def calculate_mean_distance(polygon):
    center = polygon.centroid
    exterior_coords = polygon.exterior.coords
    distances = [center.distance(Point(x, y)) for x, y in exterior_coords]
    return np.mean(distances)

train['area'] = train['geometry'].apply(calculate_area)
train['perimeter'] = train['geometry'].apply(calculate_perimeter)
train['variance_distances_to_center'] = train['geometry'].apply(calculate_var_distance_to_center)
train['calculate_mean_distance'] = train['geometry'].apply(calculate_mean_distance)
train.drop(columns = ['geometry'])

test['area'] = test['geometry'].apply(calculate_area)
test['perimeter'] = test['geometry'].apply(calculate_perimeter)
test['variance_distances_to_center'] = test['geometry'].apply(calculate_var_distance_to_center)
test['calculate_mean_distance'] = test['geometry'].apply(calculate_mean_distance)
test.drop(columns = ['geometry'])


On fait le hot one encoding sur les features 'urban_types' et 'geography_types'

In [ ]:
urban_types = ['Dense Urban', 'Sparse Urban', 'N,A', 'Urban Slum', 'Rural', 'Industrial']
for urban_type in urban_types:
    #si l'urban type est présent dans la colonne urban_types on ajoute 1 à la colonne urban_type (qui a été ajoutée au df)
    train[urban_type]=train['urban_type'].apply(lambda x: urban_type in x).astype(int) 
    test[urban_type]=test['urban_type'].apply(lambda x: urban_type in x).astype(int) 

In [ ]:
geography_types = ['Hills', 'Farms', 'Barren Land', 'Dense Forest', 'N,A', 'Coastal', 'Lakes', 'Desert', 'Sparse Forest', 'River', 'Grass Land', 'Snow']
for geography_type in geography_types:
    train[geography_type]=train['geography_type'].apply(lambda x: geography_type in x).astype(int) 
    test[geography_type]=test['geography_type'].apply(lambda x: geography_type in x).astype(int) 

On repasse tout dans l'ordre chronologique

In [ ]:
import pandas as pd
for i in range (5):
    train[f'date{i}']=pd.to_datetime(train[f'date{i}'])
    test[f'date{i}']=pd.to_datetime(test[f'date{i}'])
    
train_sort_by_date=train[['date0','date1','date2','date3','date4']]
test_sort_by_date=test[['date0','date1','date2','date3','date4']]

In [ ]:
import pandas as pd
import numpy as np

for idx in range(5):
    train[f'date{idx}'] = pd.to_datetime(train[f'date{idx}'])

dates_df = train[[f'date{i}' for i in range(5)]]
dates_df['dates_list'] = dates_df.apply(lambda x: [x[f'date{i}'] for i in range(5)], axis=1)
dates_df['sorted_indices'] = dates_df['dates_list'].apply(lambda x: np.argsort(x))
dates_df['dates_list'] = dates_df['dates_list'].apply(lambda x: [x[i] for i in np.argsort(x)])

change_status_df = train[[f'change_status_date{i}' for i in range(5)]]
dates_df['sorted_change_status'] = change_status_df.apply(lambda x: np.array([x[f'change_status_date{i}'] for i in range(5)]), axis=1)
dates_df['sorted_change_status'] = dates_df.apply(lambda x: x['sorted_change_status'][x['sorted_indices']], axis=1)

# Sort image mean values for green and red channels
green_mean_df = train[[f'img_green_mean_date{i}' for i in range(1, 6)]]
red_mean_df = train[[f'img_red_mean_date{i}' for i in range(1, 6)]]
dates_df['sorted_green_mean'] = green_mean_df.apply(lambda x: np.array([x[f'img_green_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_df['sorted_red_mean'] = red_mean_df.apply(lambda x: np.array([x[f'img_red_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_df['sorted_green_mean'] = dates_df.apply(lambda x: x['sorted_green_mean'][x['sorted_indices']], axis=1)
dates_df['sorted_red_mean'] = dates_df.apply(lambda x: x['sorted_red_mean'][x['sorted_indices']], axis=1)

# Update the original DataFrame with sorted data
for i in range(5):
    train[f'date{i}'] = dates_df['dates_list'].apply(lambda x: x[i])
    train[f'change_status_date{i}'] = dates_df['sorted_change_status'].apply(lambda x: x[i])
    train[f'img_green_mean_date{i+1}'] = dates_df['sorted_green_mean'].apply(lambda x: x[i])
    train[f'img_red_mean_date{i+1}'] = dates_df['sorted_red_mean'].apply(lambda x: x[i])


In [ ]:
import pandas as pd
import numpy as np

for idx in range(5):
    test[f'date{idx}'] = pd.to_datetime(test[f'date{idx}'])

dates_test_df = test[[f'date{i}' for i in range(5)]]
dates_test_df['dates_list'] = dates_test_df.apply(lambda x: [x[f'date{i}'] for i in range(5)], axis=1)
dates_test_df['sorted_indices'] = dates_test_df['dates_list'].apply(lambda x: np.argsort(x))
dates_test_df['dates_list'] = dates_test_df['dates_list'].apply(lambda x: [x[i] for i in np.argsort(x)])

change_status_test_df = test[[f'change_status_date{i}' for i in range(5)]]
dates_test_df['sorted_change_status'] = change_status_test_df.apply(lambda x: np.array([x[f'change_status_date{i}'] for i in range(5)]), axis=1)
dates_test_df['sorted_change_status'] = dates_test_df.apply(lambda x: x['sorted_change_status'][x['sorted_indices']], axis=1)

# Sort image mean values for green and red channels
green_mean_test_df = test[[f'img_green_mean_date{i}' for i in range(1, 6)]]
red_mean_test_df = test[[f'img_red_mean_date{i}' for i in range(1, 6)]]
dates_test_df['sorted_green_mean'] = green_mean_test_df.apply(lambda x: np.array([x[f'img_green_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_test_df['sorted_red_mean'] = red_mean_test_df.apply(lambda x: np.array([x[f'img_red_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_test_df['sorted_green_mean'] = dates_test_df.apply(lambda x: x['sorted_green_mean'][x['sorted_indices']], axis=1)
dates_test_df['sorted_red_mean'] = dates_test_df.apply(lambda x: x['sorted_red_mean'][x['sorted_indices']], axis=1)

# Update the original DataFrame with sorted data
for i in range(5):
    test[f'date{i}'] = dates_test_df['dates_list'].apply(lambda x: x[i])
    test[f'change_status_date{i}'] = dates_test_df['sorted_change_status'].apply(lambda x: x[i])
    test[f'img_green_mean_date{i+1}'] = dates_test_df['sorted_green_mean'].apply(lambda x: x[i])
    test[f'img_red_mean_date{i+1}'] = dates_test_df['sorted_red_mean'].apply(lambda x: x[i])




On récupère les dataframe avec les dates et les statuts dans le bon ordre

In [ ]:
import pandas as pd

def create_dummy_columns(df, date_index):
    dummy_df = pd.get_dummies(df[f'change_status_date{date_index}'])
    columns = [f'{column}_date{date_index}' for column in dummy_df.columns]
    dummy_df.columns = columns
    return dummy_df

dummy_columns = [create_dummy_columns(train, i) for i in range(5)]

train_change_status = pd.concat(dummy_columns, axis=1)

train_df = pd.concat([train, train_change_status], axis=1)

dummy_columns_test = [create_dummy_columns(test, i) for i in range(5)]

test_change_status = pd.concat(dummy_columns_test, axis=1)

test_df = pd.concat([test, test_change_status], axis=1)


On crée les nouvaux features qui calculent le temps passé entre les différentes photos prises d'un même environnement

In [ ]:
train_df['passed_time_0']=(train_df['date1']-train_df['date0']).apply(lambda x: x.total_seconds())
train_df['passed_time_1']=(train_df['date2']-train_df['date1']).apply(lambda x: x.total_seconds())
train_df['passed_time_2']=(train_df['date3']-train_df['date2']).apply(lambda x: x.total_seconds())
train_df['passed_time_3']=(train_df['date4']-train_df['date3']).apply(lambda x: x.total_seconds())

test_df['passed_time_0']=(test_df['date1']-test_df['date0']).apply(lambda x: x.total_seconds())
test_df['passed_time_1']=(test_df['date2']-test_df['date1']).apply(lambda x: x.total_seconds())
test_df['passed_time_2']=(test_df['date3']-test_df['date2']).apply(lambda x: x.total_seconds())
test_df['passed_time_3']=(test_df['date4']-test_df['date3']).apply(lambda x: x.total_seconds())

On récupère les fichiers train et test qui sont prêts à servir de base de données pour créer des modèles

In [ ]:
pd.concat([train_df,train_y],axis=1).to_csv('train_processed.csv')
pd.concat([test_df],axis=1).to_csv('test_processed.csv')

On charge ces fichiers, on met les NaN à 0 par défaut et on fait un XGBoost avec les paramètres qui fonctionnent le mieux pour nous, on enregistre le CSV de submission

In [ ]:
from xgboost import XGBClassifier
import pandas as pd
from sklearn.model_selection import train_test_split

# Charger les données
data = pd.read_csv('train_processed.csv', index_col=0)
data = data.fillna(0)
X = data.iloc[:, :-1].drop(columns=['N,A', 'Snow', 'urban_type', 'geography_type', 'change_type', 'geometry', 'date0', 'date1', 'date2', 'date3', 'date4','change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3' ,'change_status_date4'])
Y = data.iloc[:, -1]

data_test = pd.read_csv('test_processed.csv', index_col=0)
data_test = data_test.fillna(0)
X_test_p = data_test.iloc[:, :].drop(columns=['N,A', 'Snow', 'urban_type', 'geography_type', 'geometry', 'date0', 'date1', 'date2', 'date3', 'date4','change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3' ,'change_status_date4'])

# Définir les paramètres du modèle XGBoost
xgb_params = {
    'max_depth': 150,        # Profondeur maximale de l'arbre
    'learning_rate': 0.1,   # Taux d'apprentissage
    'n_estimators': 1000,    # Nombre d'arbres à entraîner
    'objective': 'multi:softmax',  # Fonction d'objectif pour une classification multiclasse
    'num_class': 6,         # Nombre de classes dans les données
    'eval_metric': 'mlogloss'     # Métrique d'évaluation
}

# Initialiser et entraîner le modèle XGBoost
xgb = XGBClassifier(**xgb_params)
xgb.fit(X,Y)

y_test = xgb.predict(X_test_p)
pred_df = pd.DataFrame(y_test, columns=['change_type'])
pred_df.to_csv("submission_LMTPE.csv", index=True, index_label='Id')